In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mlos_core.optimizers
import ConfigSpace as CS
from ConfigSpace import UniformIntegerHyperparameter
from ConfigSpace import UniformFloatHyperparameter
from ConfigSpace import CategoricalHyperparameter

In [12]:
def cost_model(index):
    performance_data = pd.read_csv('../../build/performance/performance_metrics.csv', header=None, names=["empty_reads", "non_empty_reads", "range_reads", "writes"])
    return performance_data.iloc[index:].mean().sum()

In [13]:
# Start defining a ConfigurationSpace for the Optimizer to search.
input_space = CS.ConfigurationSpace(seed=1234)

max_background_jobs=CS.Integer("max_background_jobs", (1,20), default=2)
max_subcompactions=CS.Integer("max_subcompactions", (1,20), default=1)
max_write_buffer_number=CS.Integer("max_write_buffer_number", (1,20), default=2)

max_bytes_for_level_multiplier=CS.Float("max_bytes_for_level_multiplier", (2,20.0), default=10)
level0_slowdown_writes_trigger=CS.Integer("level0_slowdown_writes_trigger", (1,50), default=20) 
# level0_stop_writes_trigger=CS.Integer("level0_stop_writes_trigger", (1,50), default=36) 

target_file_size_base=CS.Integer("target_file_size_base", (1,500 * 1048576), default=64 * 1048576)
target_file_size_multiplier=CS.Integer("target_file_size_multiplier", (1,20), default=1)

write_buffer_size=CS.Integer("write_buffer_size",(1,1024<<20),default=64<<20)
avoid_flush_during_shutdown=CS.Categorical("avoid_flush_during_shutdown", [True,False], default=True) 

max_bytes_for_level_base=CS.Integer("max_bytes_for_level_base",(1,1000 * 1048576),default=256 * 1048576)
level0_file_num_compaction_trigger=CS.Integer("level0_file_num_compaction_trigger",(0,200),default=4)

In [14]:
input_space.add_hyperparameters([max_background_jobs,
                                 max_subcompactions,
                                 max_write_buffer_number,
                                 max_bytes_for_level_multiplier,
                                 level0_slowdown_writes_trigger,
#                                  level0_stop_writes_trigger])
                                target_file_size_base,
                                 target_file_size_multiplier,
                                 avoid_flush_during_shutdown,
                                write_buffer_size,
                                max_bytes_for_level_base,
                                level0_file_num_compaction_trigger])

[max_background_jobs, Type: UniformInteger, Range: [1, 20], Default: 2,
 max_subcompactions, Type: UniformInteger, Range: [1, 20], Default: 1,
 max_write_buffer_number, Type: UniformInteger, Range: [1, 20], Default: 2,
 max_bytes_for_level_multiplier, Type: UniformFloat, Range: [2.0, 20.0], Default: 10.0,
 level0_slowdown_writes_trigger, Type: UniformInteger, Range: [1, 50], Default: 20,
 target_file_size_base, Type: UniformInteger, Range: [1, 524288000], Default: 67108864,
 target_file_size_multiplier, Type: UniformInteger, Range: [1, 20], Default: 1,
 avoid_flush_during_shutdown, Type: Categorical, Choices: {True, False}, Default: True,
 write_buffer_size, Type: UniformInteger, Range: [1, 1073741824], Default: 67108864,
 max_bytes_for_level_base, Type: UniformInteger, Range: [1, 1048576000], Default: 268435456,
 level0_file_num_compaction_trigger, Type: UniformInteger, Range: [0, 200], Default: 4]

In [15]:
#optimizer = mlos_core.optimizers.RandomOptimizer(parameter_space=input_space)

#optimizer = mlos_core.optimizers.FlamlOptimizer(parameter_space=input_space)

optimizer = mlos_core.optimizers.SmacOptimizer(parameter_space=input_space) # , seed=42, n_random_init=20)

In [16]:
optimizer

SmacOptimizer(space_adapter=None)

In [17]:
import os
import time
import sys
def pass_values_to_interface(new_val):
    pipe_path = '../../build/passing_params_pipe'
    if not os.path.exists(pipe_path):
        os.mkfifo(pipe_path)
    pipe_fd = os.open(pipe_path, os.O_RDWR)
    data =''
    try:
        for x in new_val:
            data+='\n' + x + '=' + str(new_val[x].iloc[0])
        data=data.encode('utf-8')
        os.write(pipe_fd, data)
        sys.stdout.flush()
    except KeyboardInterrupt:
        pass
#     finally:
#         os.close(pipe_fd)


In [18]:
def read_epochs_pipe():
    while True:
        epochs_pipe_path = '../../build/passing_epochs'
        if os.path.exists(epochs_pipe_path):
            epochs_pipe_fd = os.open(epochs_pipe_path, os.O_RDONLY)
            try:
                buffer_size = 128
                buffer = os.read(epochs_pipe_fd, buffer_size).decode('utf-8')
                if buffer:
                    reported_epochs = int(buffer)
                    print(f"Reported epochs: {reported_epochs}")
                    return reported_epochs
            except OSError as e:
                if e.errno == os.errno.EAGAIN or e.errno == os.errno.EWOULDBLOCK:
                    pass
                else:
                    raise
            finally:
                os.close(epochs_pipe_fd)
        else:
            pass
    


In [20]:
def run_optimization():
    # get a new config value suggestion to try from the optimizer.
    suggested_value = optimizer.suggest()
    pass_values_to_interface(suggested_value)
    index=read_epochs_pipe()
    time.sleep(5)
    if index==-1:
        target_value=float('inf')
    else:
        target_value = cost_model(index)
    optimizer.register(suggested_value, pd.Series([target_value]))


n_iterations = 50
for i in range(n_iterations):
    print("epoch:",i)
    run_optimization()

epoch: 0
Reported epochs: 3278
epoch: 1
Reported epochs: 3291
epoch: 2
Reported epochs: 3303
epoch: 3
Reported epochs: 3318
epoch: 4
Reported epochs: 3335
epoch: 5
Reported epochs: 3354
epoch: 6
Reported epochs: 3377
epoch: 7
Reported epochs: 3396
epoch: 8
Reported epochs: 3423
epoch: 9
Reported epochs: 3444
epoch: 10
Reported epochs: 3473
epoch: 11
Reported epochs: 3496
epoch: 12
Reported epochs: 3527
epoch: 13
Reported epochs: 3550
epoch: 14
Reported epochs: 3579
epoch: 15
Reported epochs: 3603
epoch: 16
Reported epochs: 3631
epoch: 17
Reported epochs: 3656
epoch: 18
Reported epochs: 3686
epoch: 19
Reported epochs: 3717
epoch: 20
Reported epochs: 3756
epoch: 21
Reported epochs: 3795
epoch: 22
Reported epochs: 3833
epoch: 23
Reported epochs: 3872
epoch: 24
Reported epochs: 3912
epoch: 25
Reported epochs: 3953
epoch: 26
Reported epochs: 3994
epoch: 27
Reported epochs: 4035
epoch: 28
Reported epochs: 4076
epoch: 29
Reported epochs: 4117
epoch: 30
Reported epochs: 4158
epoch: 31
Reported

In [21]:
optimizer.get_observations()

,avoid_flush_during_shutdown,level0_file_num_compaction_trigger,level0_slowdown_writes_trigger,max_background_jobs,max_bytes_for_level_base,max_bytes_for_level_multiplier,max_subcompactions,max_write_buffer_number,target_file_size_base,target_file_size_multiplier,write_buffer_size,score
0,False,159,31,4,949973953,12.782758,6,11,333967540,13,377747531,2305.500000
0,False,21,17,19,673577312,19.535268,11,18,423980065,16,2343073,1919.000000
0,True,111,46,9,54418164,6.854681,1,4,169046098,7,745432179,1532.250000
0,True,25,32,6,209011921,2.913697,15,19,487697222,11,195788877,1308.000000
0,False,143,5,16,631752554,14.416207,5,1,236907193,3,669394079,1142.000000
0,False,80,42,1,891444195,16.673102,20,8,64968671,9,826678389,991.000000
0,True,188,22,11,461199343,9.648560,9,14,266674635,19,487307431,832.428571
0,True,2,39,18,1032807303,8.461841,14,19,499273414,6,459534972,942.666667
0,False,116,25,8,348636170,18.281314,4,2,225843376,17,925783720,681.571429
0,False,54,35,13,92192592,11.542537,18,6,4752980,15,562982466,801.142857


In [22]:
optimizer.get_best_observation()

,avoid_flush_during_shutdown,level0_file_num_compaction_trigger,level0_slowdown_writes_trigger,max_background_jobs,max_bytes_for_level_base,max_bytes_for_level_multiplier,max_subcompactions,max_write_buffer_number,target_file_size_base,target_file_size_multiplier,write_buffer_size,score
0,True,141,28,5,762133881,13.496323,19,3,517046316,17,571094216,302.375


In [23]:
optimum_value=optimizer.suggest()
pass_values_to_interface(optimum_value)

In [24]:
print(optimum_value)

   avoid_flush_during_shutdown  level0_file_num_compaction_trigger  \
0                         True                                 142   

   level0_slowdown_writes_trigger  max_background_jobs  \
0                              33                    5   

   max_bytes_for_level_base  max_bytes_for_level_multiplier  \
0                 764545077                       13.496323   

   max_subcompactions  max_write_buffer_number  target_file_size_base  \
0                  20                        3              517046316   

   target_file_size_multiplier  write_buffer_size  
0                           20          567098226  


In [ ]:
hyper_parameters_integer = {
    "max_open_files":[-1,10000, -1],
    "max_total_wal_size":[0, 100000, 0],
    "delete_obsolete_files_period_micros":[0, 6 * 60 * 60 * 1000000, 6 * 60 * 60 * 1000000],
    "max_background_jobs":[2, 10, 2],
    "max_subcompactions":[1, 100, 1],
    "compaction_readahead_size":[0, 1000000, 0],
    "writable_file_max_buffer_size":[0, 1024 * 1024, 1024 * 1024],
    "delayed_write_rate":[0, 1000000, 0],
    
    "max_write_buffer_number":[1, 10, 2],
    "inplace_update_num_locks":[0, 10000, 0],
    "memtable_huge_page_size":[0, 1000000, 0],
    "arena_block_size":[0, 1000000, 0],
    "level0_slowdown_writes_trigger":[0, 100, 20],
    "level0_stop_writes_trigger":[0, 100, 36],
    "target_file_size_base":[0, 100000000, 64 * 1048576],
    "target_file_size_multiplier":[0, 100, 1],
    "max_compaction_bytes":[0, 1000000000, 64 * 1048576 * 25],
    "soft_pending_compaction_bytes_limit":[0, 1000000000, 64 * 1073741824],
    "hard_pending_compaction_bytes_limit":[0, 1000000000, 256 * 1073741824]                          
}
                                           
hyper_parameters_float ={
    "memtable_prefix_bloom_size_ratio":[0.0, 1.0, 0.0],
    "max_bytes_for_level_multiplier":[0.0, 100.0, 10.0]
}
                                           
hyper_parameters_catagorical ={
    "avoid_flush_during_shutdown":[[True, False],False],  
    "memtable_whole_key_filtering":[[True, False],False],
   "compression_per_level":[["kNoCompression", "kSnappyCompression", "kZlibCompression", "kBZip2Compression", "kLZ4Compression", "kLZ4HCCompression", "kXpressCompression", "kZSTD"], "kNoCompression"]

}